In [4]:
import gbasis
import subprocess
import numpy as np
import psi4
import chemtools
from iodata import load_one
from pyscf import gto, scf
from pyscf.tools import cubegen
from time import perf_counter
from gbasis.wrappers import from_iodata
from gbasis.evals.eval import evaluate_basis

def time_func(func):
    def wrapper(*args, **kwargs):
        start = perf_counter()
        for _ in range(5):
            func(*args, **kwargs)
        end = perf_counter()
        result = end - start
        print(f'1000 iters took {result} seconds')
        return func(*args, **kwargs)
    return wrapper

np.random.seed(123)

# Evaluation of MOs

In [6]:
# get the mols
mol_chemtools = chemtools.Molecule.from_file('water_neutral.fchk')

mol_pyscf = gto.M(
    atom = 'O 0.01086593, 0.00805023, -0.00568808; H 0.52133765,  1.67452412,  0.47604091; H 1.13869228, -0.44555963, -1.34435117',
    basis = 'CC-pVTZ'
)
mf = scf.RHF(mol_pyscf).run()

# get the points
grid_3d = np.random.random((100,3))

In [3]:
# pyscf AO
@time_func
def pyscf_ao_calc():
    global mol_pyscf, grid_3d
    ao = mol_pyscf.eval_gto('GTOval_sph', grid_3d)
    mo = ao.dot(mf.mo_coeff)
    return mo
pyscf_ao = pyscf_ao_calc()

# gbasis AO
@time_func
def gbasis_ao_calc():
    global mol_chemtools, grid_3d
    mo = mol_chemtools.compute_molecular_orbital(grid_3d)
    return mo
gbasis_ao = gbasis_ao_calc()

print(f'pyscf output shape: {pyscf_ao.shape}')
print(f'gbasis output shape: {gbasis_ao.shape}')

1000 iters took 0.0030151749961078167 seconds
1000 iters took 0.13405422799405642 seconds
pyscf output shape: (100, 58)
gbasis output shape: (58, 100)


# Gradients 

In [4]:
@time_func
def pyscf_ao_calc():
    global mol_pyscf, grid_3d
    ao = mol_pyscf.eval_gto('GTOval_sph_deriv1', grid_3d)
    mo = ao.dot(mf.mo_coeff)
    return mo
pyscf_ao = pyscf_ao_calc()

@time_func
def gbasis_ao_calc():
    global mol_chemtools, grid_3d
    mo = mol_chemtools.compute_gradient(grid_3d)
    return mo
gbasis_ao = gbasis_ao_calc()

print(f'pyscf output shape: {pyscf_ao.shape}')
print(f'gbasis output shape: {gbasis_ao.shape}')

1000 iters took 0.021943305007880554 seconds
1000 iters took 1.2868181510129943 seconds
pyscf output shape: (4, 100, 58)
gbasis output shape: (100, 3)


# Hessian

In [5]:
@time_func
def pyscf_ao_calc():
    global mol_pyscf, grid_3d
    ao = mol_pyscf.eval_gto('GTOval_sph_deriv2', grid_3d)
    mo = ao.dot(mf.mo_coeff)
    return mo
pyscf_ao = pyscf_ao_calc()

@time_func
def gbasis_ao_calc():
    global mol_chemtools, grid_3d 
    mo = mol_chemtools.compute_hessian(grid_3d)
    return mo
gbasis_ao = gbasis_ao_calc()

print(f'pyscf output shape: {pyscf_ao.shape}')
print(f'gbasis output shape: {gbasis_ao.shape}')

1000 iters took 0.03795489398180507 seconds
1000 iters took 4.804126080998685 seconds
pyscf output shape: (10, 100, 58)
gbasis output shape: (100, 3, 3)


# PySCF cubegen module

In [6]:
# pyscf has only 3 properties that can be outputed in .cube format 

def pyscf_dens(dim_xyz=100):
    outfile = 'out'
    cubegen.density(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])

def pyscf_esp(dim_xyz=100):
    outfile = 'out'
    cubegen.mep(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])

def pyscf_orb(dim_xyz=100):
    outfile = 'out'
    cubegen.orbital(mol_pyscf, outfile, dm, dim_xyz, dim_xyz, dim_xyz)
    subprocess.run(["rm", "out"])


In [7]:
import numpy
from pyscf import gto
from pyscf.dft import numint

'''
Evaluate AO functions on given grid coordinates.
'''

mol = gto.M(
    verbose = 0,
    atom = '''
    o    0    0.       0.
    h    0    -0.757   0.587
    h    0    0.757    0.587''',
    basis = '6-31g')

# Uniform grids
coords = []
for ix in numpy.arange(-10, 10, 1.):
    for iy in numpy.arange(-10, 10, 1.):
        for iz in numpy.arange(-10, 10, 1.):
            coords.append((ix,iy,iz))
coords = numpy.array(coords)

# AO value
ao_value = numint.eval_ao(mol, coords)
print('13 basis, 8000 xyz  %s' % str(ao_value.shape))

# AO value and its gradients

@time_func
def val_grad():
    ao_value = numint.eval_ao(mol, grid_3d, deriv=1)
    # print('13 basis, 8000 xyz  %s' % str(ao_value.shape))
    return ao_value.shape
val_grad()

13 basis, 8000 xyz  (8000, 13)
1000 iters took 0.001278826006455347 seconds


(4, 100, 13)